In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
import time as T
import pyproj

In [2]:
df = pd.read_csv('data.csv')
df

,ID,user_id,latitude,longitude,flag,time
0,3865408,0,30.364965,-81.675575,True,2022-09-21 02:10:55
1,3865411,0,30.364965,-81.675575,True,2022-09-21 02:11:44
2,3865608,0,30.450003,-81.662385,True,2022-09-25 21:38:08
3,3865609,0,30.450003,-81.662385,True,2022-09-25 22:02:49
4,3865610,0,30.450000,-81.662400,True,2022-09-25 22:03:38
...,...,...,...,...,...,...
7659396,13980039,34497,29.381240,-82.233210,True,2022-10-29 20:39:33
7659397,13980040,34497,29.381230,-82.233220,True,2022-10-29 20:39:40
7659398,13980047,34497,29.381230,-82.233250,True,2022-10-30 20:39:30
7659399,13980048,34497,29.381230,-82.233220,True,2022-10-30 20:39:39


In [3]:
sp = pd.read_csv('staypoints.csv')
sp

,id,user_id,started_at,finished_at,geom
0,0,0,2022-10-10 18:41:33-04:00,2022-10-10 18:54:04-04:00,POINT (-81.6625549999999976 30.4599150000000023)
1,1,2,2022-09-06 22:09:29-04:00,2022-09-06 22:19:08-04:00,POINT (-81.6582657950000055 29.6534941799999991)
2,2,2,2022-09-06 22:19:08-04:00,2022-09-06 22:22:07-04:00,POINT (-81.6840639999999922 29.6517550000000014)
3,3,2,2022-09-06 22:22:07-04:00,2022-09-06 22:31:37-04:00,POINT (-81.6582655899999992 29.6534943599999998)
4,4,2,2022-09-06 22:31:37-04:00,2022-09-06 22:34:49-04:00,POINT (-81.6840650000000181 29.6517550000000014)
...,...,...,...,...,...
397678,397678,34483,2022-10-27 03:34:53-04:00,2022-10-27 03:37:40-04:00,POINT (-81.6483199999999982 30.3217199999999991)
397679,397679,34483,2022-10-27 05:30:52-04:00,2022-10-27 05:38:14-04:00,POINT (-81.6525199999999955 30.3216099999999997)
397680,397680,34483,2022-10-27 05:38:14-04:00,2022-10-27 05:53:02-04:00,POINT (-81.6508199999999960 30.3522199999999991)
397681,397681,34484,2022-10-28 00:47:04-04:00,2022-10-28 00:55:00-04:00,POINT (-81.6688213850000011 30.3685913700000008)


In [4]:
ID_list=sp['user_id'].unique()
len(ID_list)

10987

In [5]:
ndf = df[df['user_id'].isin(ID_list)]
ndf


,ID,user_id,latitude,longitude,flag,time
0,3865408,0,30.364965,-81.675575,True,2022-09-21 02:10:55
1,3865411,0,30.364965,-81.675575,True,2022-09-21 02:11:44
2,3865608,0,30.450003,-81.662385,True,2022-09-25 21:38:08
3,3865609,0,30.450003,-81.662385,True,2022-09-25 22:02:49
4,3865610,0,30.450000,-81.662400,True,2022-09-25 22:03:38
...,...,...,...,...,...,...
7659261,13972524,34484,30.492232,-81.640842,True,2022-10-28 00:55:55
7659262,13972525,34484,30.334303,-81.668737,True,2022-10-28 01:54:30
7659269,13974593,34487,30.347470,-83.077570,True,2022-10-23 22:07:37
7659270,13974594,34487,30.347480,-83.077550,True,2022-10-23 22:07:47


In [6]:
ndf.to_csv('newData.csv')

<h3>Add columns Lat-3856 and Lon-3857 with EPSG:3857</h3>

In [7]:
# Create a transformer object to convert from EPSG 4326 (WGS84) to EPSG 3857 (Web Mercator)
transformer = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

# Iterate over rows and apply the transformer to convert the coordinates
for idx, row in tqdm(ndf.iterrows(), total=ndf.shape[0]):
    lat, lon = row['latitude'], row['longitude']
    x, y = transformer.transform(lon, lat)
    ndf.at[idx, 'Lat-3857'] = y
    ndf.at[idx, 'Lon-3857'] = x

# Print the updated DataFrame
print(ndf)

  0%|          | 0/6890198 [00:00<?, ?it/s]/var/folders/vy/m0tbb9cx2m176wk361h4ns900000gn/T/ipykernel_10470/1837106994.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf.at[idx, 'Lat-3857'] = y
/var/folders/vy/m0tbb9cx2m176wk361h4ns900000gn/T/ipykernel_10470/1837106994.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf.at[idx, 'Lon-3857'] = x
100%|██████████| 6890198/6890198 [12:15<00:00, 9371.94it/s] 

               ID  user_id   latitude  longitude  flag                 time  \
0         3865408        0  30.364965 -81.675575  True  2022-09-21 02:10:55   
1         3865411        0  30.364965 -81.675575  True  2022-09-21 02:11:44   
2         3865608        0  30.450003 -81.662385  True  2022-09-25 21:38:08   
3         3865609        0  30.450003 -81.662385  True  2022-09-25 22:02:49   
4         3865610        0  30.450000 -81.662400  True  2022-09-25 22:03:38   
...           ...      ...        ...        ...   ...                  ...   
7659261  13972524    34484  30.492232 -81.640842  True  2022-10-28 00:55:55   
7659262  13972525    34484  30.334303 -81.668737  True  2022-10-28 01:54:30   
7659269  13974593    34487  30.347470 -83.077570  True  2022-10-23 22:07:37   
7659270  13974594    34487  30.347480 -83.077550  True  2022-10-23 22:07:47   
7659271  13974595    34487  30.347040 -82.999330  True  2022-10-23 22:11:40   

             Lat-3857      Lon-3857  
0        3.55

In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
def home_location(input_df,s_hour=22,t_hour=6):
    '''
    ndf: gps data
    s_hour: define start time of night, defult = 22
    t_hour: define end time of night, defult = 6
    '''
    input_df['time']=pd.to_datetime(input_df['time'], infer_datetime_format=True)
    input_df['date']=input_df['time'].dt.date
    input_df['hour']=pd.Series(input_df['time']).dt.hour
    
    input_df['LAT_Grid']=np.round(input_df['Lat-3857']/20)*20  # Grid size = 20m
    input_df['LON_Grid']=np.round(input_df['Lon-3857']/20)*20
    id=input_df.iloc[0,0]
    input_df=input_df[(input_df['hour']>=s_hour)|(input_df['hour']<t_hour)]

    if len(input_df)==0:  # if no signal during the night, return NaN
        return np.nan,np.nan
    home = input_df.groupby(['LAT_Grid','LON_Grid'])['date'].nunique().idxmax()
    return home[0],home[1]

In [9]:
home=pd.DataFrame(columns=['ID','LAT','LON']) # initial a DataFrame to save home location

for i in tqdm(ID_list[0:]): 
    dffh=ndf[ndf['user_id']==i].sort_values('time',axis=0,ascending=True) # extract records
    h_lat,h_lon=home_location(dffh) # calculate home location
    home=home.append(pd.DataFrame([[i,h_lat,h_lon]],columns=['ID','LAT','LON'])) # append the new row 

100%|██████████| 10987/10987 [02:02<00:00, 89.35it/s]


In [10]:
nan_df = home[home.isna().any(axis=1)]
home.dropna(subset=['LAT'], inplace=True)
home

,ID,LAT,LON
0,0,3561520.0,-9090620.0
0,2,3458860.0,-9093020.0
0,3,3456360.0,-9163720.0
0,8,3510020.0,-9105740.0
0,10,3565080.0,-9087560.0
...,...,...,...
0,34472,3544960.0,-9078280.0
0,34481,3530840.0,-9102880.0
0,34483,3503620.0,-9058160.0
0,34484,3546600.0,-9091320.0


<h3>----------------------------------------------------------------------------------------------------------------</h3>

<h3>
Further adjust to infer home location for more users</h3>

In [11]:
ID_list=nan_df['ID'].unique()
len(ID_list)

1558

In [12]:
#check if a date is in saturday or sunday.
def is_weekend(date):
    return date.weekday() in [5, 6]

def home_location(input_df):
    '''
    ndf: gps data
    '''
    input_df['time']=pd.to_datetime(input_df['time'], infer_datetime_format=True)
    input_df['date']=input_df['time'].dt.date
    input_df['hour']=pd.Series(input_df['time']).dt.hour
    
    input_df['LAT_Grid']=np.round(input_df['Lat-3857']/20)*20  # Grid size = 20m
    input_df['LON_Grid']=np.round(input_df['Lon-3857']/20)*20
    input_df= input_df[input_df['date'].apply(is_weekend)]
    
    if len(input_df)==0: 
        return np.nan,np.nan
    home2 = input_df.groupby(['LAT_Grid','LON_Grid']).count()['ID'].idxmax()
    return home2[0],home2[1]

In [13]:
home2=pd.DataFrame(columns=['ID','LAT','LON']) # initial a DataFrame to save home location

for i in tqdm(ID_list[0:]): 
    dffh=ndf[ndf['user_id']==i].sort_values('time',axis=0,ascending=True) # extract records
    h_lat,h_lon=home_location(dffh) # calculate home location
    home2=home2.append(pd.DataFrame([[i,h_lat,h_lon]],columns=['ID','LAT','LON'])) # append the new row 

100%|██████████| 1558/1558 [00:16<00:00, 94.79it/s]


In [14]:
home2.dropna(subset=['LAT'], inplace=True)
home2

,ID,LAT,LON
0,82,3373740.0,-9074520.0
0,83,3446280.0,-9044880.0
0,105,3513240.0,-9105460.0
0,243,3382800.0,-9050540.0
0,270,3565500.0,-9092600.0
...,...,...,...
0,34249,3373140.0,-9053960.0
0,34319,3535180.0,-9080400.0
0,34321,3380620.0,-9051940.0
0,34355,3383780.0,-9158680.0


In [15]:
new_home = pd.concat([home, home2], ignore_index=True)
new_home

,ID,LAT,LON
0,0,3561520.0,-9090620.0
1,2,3458860.0,-9093020.0
2,3,3456360.0,-9163720.0
3,8,3510020.0,-9105740.0
4,10,3565080.0,-9087560.0
...,...,...,...
10201,34249,3373140.0,-9053960.0
10202,34319,3535180.0,-9080400.0
10203,34321,3380620.0,-9051940.0
10204,34355,3383780.0,-9158680.0


In [16]:
# Create a transformer object to convert from EPSG 3857 (Web Mercator) to EPSG 4326 (WGS84)
inputGrid = pyproj.Proj(projparams='epsg:3857')
wgs84 = pyproj.Proj(projparams='epsg:4326')

new_home = new_home.reset_index(drop=True)

# Iterate over rows and apply the transformer to convert the coordinates
for idx, row in tqdm(new_home.iterrows(), total=new_home.shape[0]):
    lat, lon = pyproj.transform(inputGrid, wgs84, row['LON'], row['LAT'])
    new_home.at[idx, 'LAT-4326'] = lat
    new_home.at[idx, 'LON-4326'] = lon

# Print the updated DataFrame
new_home

100%|██████████| 10206/10206 [00:17<00:00, 598.60it/s]


,ID,LAT,LON,LAT-4326,LON-4326
0,0,3561520.0,-9090620.0,30.449959,-81.662429
1,2,3458860.0,-9093020.0,29.651722,-81.683988
2,3,3456360.0,-9163720.0,29.632203,-82.319097
3,8,3510020.0,-9105740.0,30.050323,-81.798254
4,10,3565080.0,-9087560.0,30.477524,-81.634940
...,...,...,...,...,...
10201,34249,3373140.0,-9053960.0,28.980309,-81.333106
10202,34319,3535180.0,-9080400.0,30.245766,-81.570621
10203,34321,3380620.0,-9051940.0,29.039072,-81.314961
10204,34355,3383780.0,-9158680.0,29.063888,-82.273822


In [17]:
new_home.to_csv('home_location.csv')